# Lab1: Set Covering

# Librairies

In [53]:
import numpy as np
import logging
import random

# Problem

## Definition of the problem

- **Input**: an integer k, a set P and S a subset of parts of P
- **Problem**: There is a subset T of S, such that each number between 0 and k-1 appears in at least one list and the total numbers of elements in all T is minimum 

## Interpretation

- According to many research, set covering problem is the equivalent of a hitting graph problem, where the edges represents the elements of our goal and the nodes the set
- It is a  graph problem

## Modelisation

- node: list L
- edge: len(L)
- goal: set(range(0:N))

## General code

In [54]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Example Github: Greedy Algorithm

In [55]:
def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    n=0
    i=0
    while goal != covered:
        x = all_lists.pop(0)
        i = i+1
        if not set(x) < covered:
            n = n+1
            solution.append(x)
            covered |= set(x)

    logging.info(f"NEW SOL: Found a solution in {i:,} steps; visited {n:,} nodes")
    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000,2000]:
    greedy(N)



NEW SOL: Found a solution in 13 steps; visited 5 nodes
Greedy solution for N=5: w=5 (bloat=0%)
NEW SOL: Found a solution in 14 steps; visited 7 nodes
Greedy solution for N=10: w=13 (bloat=30%)
NEW SOL: Found a solution in 14 steps; visited 12 nodes
Greedy solution for N=20: w=46 (bloat=130%)
NEW SOL: Found a solution in 23 steps; visited 19 nodes
Greedy solution for N=100: w=332 (bloat=232%)
NEW SOL: Found a solution in 28 steps; visited 24 nodes
Greedy solution for N=500: w=2162 (bloat=332%)
NEW SOL: Found a solution in 27 steps; visited 26 nodes
Greedy solution for N=1000: w=4652 (bloat=365%)
NEW SOL: Found a solution in 36 steps; visited 34 nodes
Greedy solution for N=2000: w=12169 (bloat=508%)


## Generic Search

### Classes

In [56]:
class State:
    """
    The state at a moment of the search
    defined by an array
    """    
    def __init__(self, arr=[]):
       self._array = arr.copy()
    
    def __hash__(self):
        return hash(np.array(self._array).tobytes())
    
    def __eq__(self, other):
        return sum(len(_) for _ in self._array) ==  sum(len(_) for _ in other._array)

    def __lt__(self, other):
        return sum(len(_) for _ in self._array) < sum(len(_) for _ in other._array)
    def __str__(self):
        return str(self) 
    
    @property
    def array(self):
        return self._array
    @array.setter
    def array(self,value):
        self._array = value

In [57]:
def goal_test(state,goal):
    return state == goal
def possible_actions(all_lists,covering):
    return sorted([array for array in all_lists if len(set(array).difference(covering)) > 0], key=lambda l: len(l) )  
def result(state, a):
    array = state.array.copy()
    array.append(a)
    return(State(array))

In [67]:

from gx_utils  import PriorityQueue

def possible_actionsV2(all_lists,covering):
    return [array for array in all_lists if len(set(array).difference(covering)) > 0]

logging.basicConfig(format="%(message)s", level=logging.INFO)
def searchV2(N,initial_state,goal_test,parent_state,state_cost,priority_function,unit_cost):
    """
    The search function
    """
    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0
    solutions = []
    covering = set()
    step = 0
    all_lists = sorted(problem(N,seed=42), key=lambda l: len(l))
    while state is not None and  covering != goal_test:
        frontier = PriorityQueue() # My frontier is 0 at each loop because he consider all the possible connected wiht the actual state. 
        for a in possible_actionsV2(all_lists,covering):
                new_state = result(state, a)
                cost = unit_cost(new_state) 
                if new_state not in state_cost and new_state not in frontier:
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    frontier.push(new_state, p=priority_function(new_state))
                elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                    old_cost = state_cost[new_state]
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
        if frontier:
            state = frontier.pop()
            step = step + 1
            solutions.append(state.array)
            covering |= set([el for a in state.array for el in a])
        else:
            state = None
    path = list()
    s = state
    i = 0
    while s:
        path.append(s)
        s = parent_state[s]
    logging.info(f"NEW SOL: Found a solution in {step} nodes; ")
    return state.array,list(reversed(path))

### Gready Best-First

In [70]:
def greedy_first(N):
    INITIAL_STATE = State()
    parent_state = dict()
    state_cost = dict()
    covering = set()
    goal  =  set(range(N))
    def h(state):
        return len(goal.difference(set([el for a in state.array for el in a])))

    final = searchV2(
        N,
        INITIAL_STATE,
        goal_test=goal,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: h(s) ,
        unit_cost=lambda a: sum(len(_) for _ in a.array)
    )
    logging.info(
            f"gready_best_first first solution for N={N}: w={sum(len(_) for _  in final[0])} (bloat={(sum(len(_) for _ in final[0])-N)/N*100:.0f}%)"
        )
logging.getLogger().setLevel(logging.INFO)
for N in [5,10,20,100,500,1000]:
    greedy_first(N)

C:\Users\thoma\AppData\Local\Temp/ipykernel_26064/2412138750.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return hash(np.array(self._array).tobytes())
NEW SOL: Found a solution in 3 nodes; 
gready_best_first first solution for N=5: w=5 (bloat=0%)
NEW SOL: Found a solution in 3 nodes; 
gready_best_first first solution for N=10: w=10 (bloat=0%)
NEW SOL: Found a solution in 4 nodes; 
gready_best_first first solution for N=20: w=28 (bloat=40%)
NEW SOL: Found a solution in 5 nodes; 
gready_best_first first solution for N=100: w=192 (bloat=92%)
NEW SOL: Found a solution in 7 nodes; 
gready_best_first first solution for N=500: w=1320 (bloat=164%)
NEW SOL: Found a solution in 8 nodes; 
gready_best_first first solution for N=1000: w=2893 (bloat=189%)


### A*

In [69]:
def a_star(N):
    INITIAL_STATE = State()
    parent_state = dict()
    state_cost = dict()
    covering = set()
    goal  =  set(range(N))
    def h(state):
        return len(goal.difference(set([el for a in state.array for el in a])))

    final = searchV2(
        N,
        INITIAL_STATE,
        goal_test=goal,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: h(s) + state_cost[s] ,
        unit_cost=lambda a: sum(len(_) for _ in a.array)
    )
    logging.info(
            f"A* first solution for N={N}: w={sum(len(_) for _  in final[0])} (bloat={(sum(len(_) for _ in final[0])-N)/N*100:.0f}%)"
        )
logging.getLogger().setLevel(logging.INFO)
for N in [5,10,20,100,500,1000]:
    a_star(N)

C:\Users\thoma\AppData\Local\Temp/ipykernel_26064/2412138750.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return hash(np.array(self._array).tobytes())
NEW SOL: Found a solution in 5 nodes; 
A* first solution for N=5: w=5 (bloat=0%)
NEW SOL: Found a solution in 6 nodes; 
A* first solution for N=10: w=11 (bloat=10%)
NEW SOL: Found a solution in 7 nodes; 
A* first solution for N=20: w=28 (bloat=40%)
NEW SOL: Found a solution in 13 nodes; 
A* first solution for N=100: w=230 (bloat=130%)
NEW SOL: Found a solution in 20 nodes; 
A* first solution for N=500: w=1828 (bloat=266%)
NEW SOL: Found a solution in 23 nodes; 
A* first solution for N=1000: w=4130 (bloat=313%)


### Breath First

In [63]:
def breath_first(N):
    INITIAL_STATE = State()
    parent_state = dict()
    state_cost = dict()
    covering = set()
    goal  =  set(range(N))


    final = searchV2(
        N,
        INITIAL_STATE,
        goal_test=goal,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost) ,
        unit_cost=lambda a: 1
    )
    logging.info(
            f"breath_first first solution for N={N}: w={sum(len(_) for _  in final[0])} (bloat={(sum(len(_) for _ in final[0])-N)/N*100:.0f}%)"
        )
logging.getLogger().setLevel(logging.INFO)
for N in [5,10,20,100,500,1000]:
    breath_first(N)

C:\Users\thoma\AppData\Local\Temp/ipykernel_26064/2412138750.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return hash(np.array(self._array).tobytes())
NEW SOL: Found a solution in 6 steps; visited 40 nodes
breath_first first solution for N=5: w=5 (bloat=0%)
NEW SOL: Found a solution in 8 steps; visited 267 nodes
breath_first first solution for N=10: w=13 (bloat=30%)
NEW SOL: Found a solution in 13 steps; visited 304 nodes
breath_first first solution for N=20: w=46 (bloat=130%)
NEW SOL: Found a solution in 20 steps; visited 7,270 nodes
breath_first first solution for N=100: w=332 (bloat=232%)
NEW SOL: Found a solution in 25 steps; visited 40,366 nodes
breath_first first solution for N=500: w=2162 (bloat=332%)
NEW SOL: Found a solution in 27 steps; visited 87,907

### Depth first

In [64]:
def depth_first(N):
    INITIAL_STATE = State()
    parent_state = dict()
    state_cost = dict()
    covering = set()
    goal  =  set(range(N))


    final = searchV2(
        N,
        INITIAL_STATE,
        goal_test=goal,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: -len(state_cost) ,
        unit_cost=lambda a: 1
    )
    logging.info(
            f"breath_first first solution for N={N}: w={sum(len(_) for _  in final[0])} (bloat={(sum(len(_) for _ in final[0])-N)/N*100:.0f}%)"
        )
logging.getLogger().setLevel(logging.INFO)
for N in [5,10,20,100,500,1000]:
    depth_first(N)

C:\Users\thoma\AppData\Local\Temp/ipykernel_26064/2412138750.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return hash(np.array(self._array).tobytes())
NEW SOL: Found a solution in 5 steps; visited 47 nodes
breath_first first solution for N=5: w=8 (bloat=60%)
NEW SOL: Found a solution in 5 steps; visited 119 nodes
breath_first first solution for N=10: w=19 (bloat=90%)
NEW SOL: Found a solution in 8 steps; visited 159 nodes
breath_first first solution for N=20: w=57 (bloat=185%)
NEW SOL: Found a solution in 10 steps; visited 3,123 nodes
breath_first first solution for N=100: w=379 (bloat=279%)
NEW SOL: Found a solution in 11 steps; visited 16,751 nodes
breath_first first solution for N=500: w=2044 (bloat=309%)
NEW SOL: Found a solution in 14 steps; visited 40,987